In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

# CSV dosyasını okuma
train_data = pd.read_csv('/content/drive/MyDrive/datathon2024/datathon-2024/train.csv', low_memory=False)

In [11]:
# Önemsiz sütunları kaldırma
drop_columns = [
    'id'
]

#, 'Spor Dalindaki Rolunuz Nedir?', 'Uye Oldugunuz Kulubun Ismi' 'Basvuru Yili', 'Cinsiyet', 'Dogum Tarihi', 'Dogum Yeri',

train_data_cleaned = train_data.drop(columns=drop_columns)
train_data_cleaned.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed_train.csv', index=False)

# Düzenlenmiş veri setini tekrar yükleme
train_data = pd.read_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed_train.csv', low_memory=False)

In [12]:
# Universite Not Ortalamasi sütununda aralıklı değerlerin ortalamasını hesaplama
def calculate_average(value):
    if isinstance(value, str) and '-' in value:
        try:
            lower, upper = value.split('-')
            average = (float(lower.strip()) + float(upper.strip())) / 2
            return average
        except ValueError:
            return None
    return value

train_data['Universite Not Ortalamasi'] = train_data['Universite Not Ortalamasi'].apply(calculate_average)
train_data['Universite Not Ortalamasi'] = pd.to_numeric(train_data['Universite Not Ortalamasi'], errors='coerce')
train_data['Universite Not Ortalamasi'] = train_data['Universite Not Ortalamasi'].fillna(train_data['Universite Not Ortalamasi'].mean())

# Düzenlenmiş veri setini kaydetme
train_data.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed1_train.csv', index=False)

In [13]:
# Eğitim verisini tekrar yükleme
train_data = pd.read_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed1_train.csv', low_memory=False)

# Sayısal veriler için ortalama ile doldurma
numeric_cols = train_data.select_dtypes(include=['float64', 'int64']).columns
numeric_imputer = SimpleImputer(strategy='mean')
train_data[numeric_cols] = numeric_imputer.fit_transform(train_data[numeric_cols])

# Kategorik veriler için en sık görülen değerle doldurma
categorical_cols = train_data.select_dtypes(include=['object']).columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
train_data[categorical_cols] = categorical_imputer.fit_transform(train_data[categorical_cols])

In [14]:
# Kategorik sütunları etiketleme ve dönüştürme
for col in categorical_cols:
    encoder = LabelEncoder()
    train_data[col] = encoder.fit_transform(train_data[col].astype(str))

# Aykırı değerleri IQR yöntemi ile tespit etme ve silme
def remove_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return data[(data[column] >= lower_bound) & (data[column] <= upper_bound)]

# Aykırı değerlerden temizleme (Örnek olarak tüm sayısal sütunlar için uygulanabilir)
for col in numeric_cols:
    train_data = remove_outliers_iqr(train_data, col)

# Çok eksik verisi olan satırların silinmesi
missing_ratio = train_data.isnull().mean(axis=1)
threshold = 0.5  # %50'den fazla eksik verisi olan satırları sil
train_data = train_data[missing_ratio < threshold]

# Eksik değerler için ileri ve geri doldurma
train_data.ffill(inplace=True)  # İleri doldurma
train_data.bfill(inplace=True)  # Geri doldurma

In [15]:
"""# Makine öğrenimi modeli ile eksik değerleri tahmin etme
target_col = 'Lise Mezuniyet Notu'  # Tahmin edilecek sütun
features = train_data.drop(columns=[target_col])

# Eksik ve eksik olmayan verilerin ayrıştırılması
train_data_non_missing = train_data.dropna(subset=[target_col])
train_data_missing = train_data[train_data[target_col].isnull()]

# Model oluşturma ve eğitme
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(train_data_non_missing.drop(columns=[target_col]), train_data_non_missing[target_col])

# Eksik değerlerin tahmin edilmesi
predictions = model.predict(train_data_missing.drop(columns=[target_col]))

# Tahmin edilen değerlerle doldurma
train_data.loc[train_data[target_col].isnull(), target_col] = predictions

# Düzenlenen veri setinin kaydedilmesi
train_data.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed2_train.csv', index=False"""

import xgboost as xgb
from sklearn.model_selection import train_test_split

# Tahmin edilecek sütun
target_col = 'Degerlendirme Puani'

# Özellikler (target_col hariç tüm sütunlar)
features = train_data.drop(columns=[target_col])

# Eksik ve eksik olmayan verilerin ayrıştırılması
train_data_non_missing = train_data.dropna(subset=[target_col])
train_data_missing = train_data[train_data[target_col].isnull()]

# Model oluşturma
model = xgb.XGBRegressor(n_estimators=100, random_state=42)

# Modeli eğitmek için eksik olmayan verileri kullanma
model.fit(train_data_non_missing.drop(columns=[target_col]), train_data_non_missing[target_col])

# Eksik değerlerin tahmin edilmesi
predictions = model.predict(train_data_missing.drop(columns=[target_col]))

# Tahmin edilen değerlerle eksik değerlerin doldurulması
train_data.loc[train_data[target_col].isnull(), target_col] = predictions

# Düzenlenen veri setinin kaydedilmesi
train_data.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed2_train.csv', index=False)

print("Eksik değerler başarıyla tahmin edildi ve veri seti kaydedildi.")

Eksik değerler başarıyla tahmin edildi ve veri seti kaydedildi.


In [17]:
#HİPERPARAMETRE
#OPTİMİZASYONU
#YAPILIYOR
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Hiperparametre aralıkları
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

# RandomizedSearchCV ile hiperparametre optimizasyonu
random_search = RandomizedSearchCV(
    estimator=xgb.XGBRegressor(random_state=42),
    param_distributions=param_grid,
    n_iter=50,        # Kaç farklı kombinasyon deneneceği
    scoring='neg_mean_squared_error',
    cv=3,             # 3 katlı çapraz doğrulama
    verbose=2,
    n_jobs=-1,        # Paralel işlem
    random_state=42
)

# Eğitim seti ile optimizasyonu gerçekleştirme
random_search.fit(X_train, y_train)

# En iyi parametreler
print("Best Parameters:", random_search.best_params_)

# En iyi modeli kullanarak eğitim seti ile yeniden eğitme
best_model = random_search.best_estimator_
best_model.fit(X_train, y_train)

# Modeli değerlendirme
y_pred_val = best_model.predict(X_val)
print("Optimized Validation MSE:", mean_squared_error(y_val, y_pred_val))

# Test veri seti üzerinde tahmin yapma
test_predictions = best_model.predict(test_data_processed)

# Tahminleri bir CSV dosyasına yazma
submission = pd.DataFrame({'id': test_data['id'], 'Degerlendirme Puani': test_predictions})
submission.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/submission_optimized.csv', index=False)

print("Tahminler başarıyla kaydedildi.")

Fitting 3 folds for each of 50 candidates, totalling 150 fits
Best Parameters: {'subsample': 1.0, 'reg_lambda': 2, 'reg_alpha': 0, 'n_estimators': 500, 'max_depth': 5, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.6}
Optimized Validation MSE: 34.48892829283832
Tahminler başarıyla kaydedildi.


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

# Veriyi yükleme
data = pd.read_csv('/content/drive/MyDrive/datathon2024/datathon-2024/processed2_train.csv')  # Ön işleme yapılmış eğitim verisi
test_data = pd.read_csv('/content/drive/MyDrive/datathon2024/datathon-2024/test_x.csv')     # Test verisi

# Özellikler ve etiketlerin ayrılması
X = data.drop(columns=['Degerlendirme Puani'])  # Özellikler
y = data['Degerlendirme Puani']                  # Etiketler

# Kategorik verileri sayısal verilere dönüştürme
def preprocess_data(df, reference_df=None):
    df = pd.get_dummies(df)
    if reference_df is not None:
        # Referans veri setindeki sütunlarla eşitle
        df = df.reindex(columns=reference_df.columns, fill_value=0)
    # Eksik verileri doldurma
    df.fillna(df.mean(), inplace=True)
    return df

# Eğitim veri setini işleme
X_processed = preprocess_data(X)

# Test veri setini işleme, eğitim veri setindeki sütunlarla eşitleme
test_data_processed = preprocess_data(test_data, reference_df=X_processed)

# Eğitim ve test veri setlerine ayırma
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Modeli oluşturma ve eğitme
model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Modeli değerlendirme (isteğe bağlı)
y_pred_val = model.predict(X_val)
print("Validation MSE:", mean_squared_error(y_val, y_pred_val))

# Test veri seti üzerinde tahmin yapma
test_predictions = model.predict(test_data_processed)

# Tahminleri bir CSV dosyasına yazma
submission = pd.DataFrame({'id': test_data['id'], 'Degerlendirme Puani': test_predictions})
submission.to_csv('/content/drive/MyDrive/datathon2024/datathon-2024/submission4.csv', index=False)

Validation MSE: 35.94034923332127


In [ ]:
"""import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Veriyi yükleme
data = pd.read_csv('C:\\Users\\90552\\Downloads\\processed2_train.csv')  # Ön işleme yapılmış eğitim verisi
test_data = pd.read_csv('C:\\Users\\90552\\Downloads\\test_x.csv')     # Test verisi

# Özellikler ve etiketlerin ayrılması
X = data.drop(columns=['Degerlendirme Puani'])  # Özellikler
y = data['Degerlendirme Puani']                  # Etiketler

# Kategorik verileri sayısal verilere dönüştürme
def preprocess_data(df, reference_df=None):
    df = pd.get_dummies(df)
    if reference_df is not None:
        # Referans veri setindeki sütunlarla eşitle
        df = df.reindex(columns=reference_df.columns, fill_value=0)
    # Eksik verileri doldurma
    df.fillna(df.mean(), inplace=True)
    return df

# Eğitim veri setini işleme
X_processed = preprocess_data(X)

# Test veri setini işleme, eğitim veri setindeki sütunlarla eşitleme
test_data_processed = preprocess_data(test_data, reference_df=X_processed)

# Eğitim ve test veri setlerine ayırma
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Modeli oluşturma ve eğitme
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Modeli değerlendirme (isteğe bağlı)
y_pred_val = model.predict(X_val)
print("Validation MSE:", mean_squared_error(y_val, y_pred_val))

# Test veri seti üzerinde tahmin yapma
test_predictions = model.predict(test_data_processed)

# Tahminleri bir CSV dosyasına yazma
submission = pd.DataFrame({'id': test_data['id'], 'Degerlendirme Puani': test_predictions})
submission.to_csv('submission.csv', index=False)"""

"""import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# Veriyi yükleme
data = pd.read_csv('C:\\Users\\90552\\Downloads\\processed2_train.csv')  # Ön işleme yapılmış eğitim verisi
test_data = pd.read_csv('C:\\Users\\90552\\Downloads\\test_x.csv')     # Test verisi

# Özellikler ve etiketlerin ayrılması
X = data.drop(columns=['Degerlendirme Puani'])  # Özellikler
y = data['Degerlendirme Puani']                  # Etiketler

# Kategorik verileri sayısal verilere dönüştürme
def preprocess_data(df, reference_df=None):
    df = pd.get_dummies(df)
    if reference_df is not None:
        # Referans veri setindeki sütunlarla eşitle
        df = df.reindex(columns=reference_df.columns, fill_value=0)
    # Eksik verileri doldurma
    df.fillna(df.mean(), inplace=True)
    return df

# Eğitim veri setini işleme
X_processed = preprocess_data(X)

# Test veri setini işleme, eğitim veri setindeki sütunlarla eşitleme
test_data_processed = preprocess_data(test_data, reference_df=X_processed)

# Eğitim ve test veri setlerine ayırma
X_train, X_val, y_train, y_val = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Modeli oluşturma ve eğitme
model = XGBRegressor()
model.fit(X_train, y_train)

# Modeli değerlendirme (isteğe bağlı)
y_pred_val = model.predict(X_val)
print("Validation MSE:", mean_squared_error(y_val, y_pred_val))

# Test veri seti üzerinde tahmin yapma
test_predictions = model.predict(test_data_processed)

# Tahminleri bir CSV dosyasına yazma
submission = pd.DataFrame({'id': test_data['id'], 'Degerlendirme Puani': test_predictions})
submission.to_csv('submission2.csv', index=False)"""

#burdan aldım kodu bu kısımdan